In [20]:
import os
from dotenv import load_dotenv
import requests
import csv
import time
import tweepy
from datetime import datetime, timedelta
from google.cloud import storage
import gcsfs
import pandas as pd

# API

In [3]:
def get_bearer_token():
    load_dotenv()
    return os.getenv('TWITTER_BEARER_TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [5]:
def create_url(user_id, start_time, end_time, max_results = 10):
    
    search_url = f"https://api.twitter.com/2/users/{user_id}/mentions" #Endpoint for user mentions, max 800 tweets

    #params fot the endpoint
    query_params = {
                    'start_time': start_time,
                    'end_time': end_time,
                    'max_results': max_results,
                    'pagination_token': {}
                    }
    return (search_url, query_params)

In [6]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['pagination_token'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [7]:
def get_user_id(user_name):
    url = f'https://api.twitter.com/2/users/by/username/{user_name}'
    response = requests.request("GET", url, headers = headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()['data']['id']

In [43]:
def append_to_csv(json_response, date):
    
    # Counter to follow the appends
    counter = 0
    
    # Open OR create the CSV file
    csvFile = open("../raw_data/tweets.csv", "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    # Loop through each tweet
    for tweet in json_response['data']:
        text = tweet['text'].replace('\n', ' ')
        row = [date, text]
        csvWriter.writerow(row) #add date and tweet to csv
        counter += 1

    # Close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print(f"Number of Tweets added from this response: {counter}") 

In [66]:
#Inputs for the request
bearer_token = get_bearer_token()
headers = create_headers(bearer_token)
user_id = get_user_id('Apple')
start_time = '2022-05-30T00:00:00.000Z'
end_time = '2022-05-31T00:00:00.000Z'

In [87]:
url, params = create_url(user_id, '2022-05-29T00:00:00.000Z', '2022-05-30T00:00:00.000Z')
json_response = connect_to_endpoint(url, headers, params)

Endpoint Response Code: 200


In [88]:
json_response

{'meta': {'result_count': 0}}

In [140]:
#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("../raw_data/tweets.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['dates','tweets'])

# Inputs
max_results = 100 # Max tweets per time period
flag = True
next_token = None
bearer_token = get_bearer_token()
headers = create_headers(bearer_token)
user_id = get_user_id('Apple')

# Dates
start_time = datetime.now() - timedelta(hours=3)
end_time = datetime.now()

# Check if flag is true
while flag:
    print("-------------------")
    print("Token: ", next_token)

    url, params = create_url(user_id, start_time, end_time, max_results)
    json_response = connect_to_endpoint(url, headers, params, next_token)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)

        if result_count is not None and result_count > 0 and next_token is not None:
            print("Start Date: ", start_time)
            append_to_csv(json_response, end_time)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)   

    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            print("Start Date: ", start_time)
            append_to_csv(json_response, start_time)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(5)

        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(5)
csvFile.close()
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 400


Exception: (400, '{"errors":[{"parameters":{"start_time":["2022-05-31 12:30:35.589751"]},"message":"The `start_time` query parameter value [2022-05-31 12:30:35.589751] is not a valid RFC3339 date-time."},{"parameters":{"end_time":["2022-05-31 15:30:35.589840"]},"message":"The `end_time` query parameter value [2022-05-31 15:30:35.589840] is not a valid RFC3339 date-time."}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}')

# tweepy Stream API v1

In [115]:
consumer_key = os.getenv('TWITTER_API_KEY')
consumer_secret = os.getenv('TWITTER_API_KEY_SECRET')
access_token = os.getenv('TWITTER_ACCESS_TOKEN')
access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

In [116]:
class SimpleListener(tweepy.Stream):
    def on_status(self, status):
        #code to run each time the stream receives a status
        print(status.text)

    def on_direct_message(self, status):
        #code to run each time the stream receives a direct message
        print('on_direct_message: status received:\n{status}')

    def on_data(self, status):
        #code to run each time you receive some data (direct message, delete, profile update, status,...)
        print(status.text)

    def on_error(self, staus_code):
        #code to run each time an error is received
        if status_code == 420:
            return False
        else:
            return True

In [117]:
stream = SimpleListener(consumer_key=consumer_key, 
                        consumer_secret=consumer_secret, 
                        access_token=access_token,
                        access_token_secret=access_token_secret)
stream.filter(track=['Apple'], languages=['en'])

time.sleep(180)
stream.disconnect()

Stream encountered HTTP error: 403


KeyboardInterrupt: 

# tweepy Get user mentions

In [11]:
client = tweepy.Client(bearer_token)
user_id = get_user_id('Apple')

start_time = datetime.now() - timedelta(hours=3)
end_time = datetime.now()

response = client.get_users_mentions(id=user_id, start_time=start_time, end_time=end_time)
response

Response(data=[<Tweet id=1531910598421798914 text='@Apple where is the AMERICAN SOLDIER emoji? You have freaking comrades (Russia’s proud), some French Foreign Legion dude, but not a single soldier, sailor or Airmen. \n\nYou even have zombies.... so don’t pretend this is not a direct reflection of how little respect you have. https://t.co/h1ewE7j8KA'>, <Tweet id=1531910527483809793 text='@Apple ⛔️Avoid HP Laptops⛔️ #HP engr destroyed fully functional laptop - damaged display &amp; left PC in 2 pieces on 11May22. Aft 21days, HP says we’ll pay for display but not repair or replace laptop.  \n\n@HP @EnriqueJLores @AlexChoHP @HPIndia @HPIndia_MD #Apple #Lenovo #Dell #ASUS #Acer https://t.co/qeGCr5zQYr'>, <Tweet id=1531910520990683137 text='2016 Vs 2022\nImagine 6 years later @Apple proves to be copy cats https://t.co/d5REhxBSaE'>, <Tweet id=1531910454015995905 text='@Apple this ad is tracking me as we speak'>, <Tweet id=1531910450203484161 text='@apple how does the fruit link with your ide

In [12]:
def append_to_csv(response, date):
    
    # Counter to follow the appends
    counter = 0
    
    # Open OR create the CSV file
    csvFile = open("../raw_data/tweets.csv", "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)
    
    # Loop through each tweet
    for tweet in response.data:
        text = tweet['text'].replace('\n', ' ')
        row = [date, text]
        csvWriter.writerow(row) #add date and tweet to csv
        counter += 1

    # Close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print(f"Number of Tweets added from this response: {counter}") 

In [13]:
%%time

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("../raw_data/tweets_playstation.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['dates','tweets'])

# Inputs
max_results = 100 # Max tweets per time period
flag = True
next_token = None
bearer_token = get_bearer_token()
headers = create_headers(bearer_token)
user_id = get_user_id('PlayStation')

# Dates
start_time = datetime.now() - timedelta(hours=3)
end_time = datetime.now()

# Check if flag is true
while flag:
    
    response = client.get_users_mentions(id=user_id, start_time=start_time, end_time=end_time)
    result_count = response.meta['result_count']

    if 'next_token' in response.meta and total_tweets<1000:
        # Save the token to use for next call
        next_token = response.meta['next_token']
        print("Next Token: ", next_token)

        if result_count is not None and result_count > 0 and next_token is not None:
            print("Start Date: ", start_time)
            append_to_csv(response, end_time)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(2)   

    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            print("Start Date: ", start_time)
            append_to_csv(response, start_time)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(2)

        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(2)
csvFile.close()
print("Total number of results: ", total_tweets)

Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  10
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  20
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  30
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  40
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  50
-------------------
Next Token:  7140dibdnow9c7btw

Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  440
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  450
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  460
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  470
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  480
-------------------
Next Token:  7140dibdnow9

Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  870
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  880
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  890
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  900
-------------------
Next Token:  7140dibdnow9c7btw421t2haiggv5trjsudubxflxjfav
Start Date:  2022-06-01 07:09:35.555145
Number of Tweets added from this response: 10
Total # of Tweets added:  910
-------------------
Next Token:  7140dibdnow9

# Upload to GCP

In [28]:
%%time

#Total number of tweets we collected from the loop
total_tweets = 0

# Inputs
max_results = 100 # Max tweets per time period
flag = True
next_token = None
bearer_token = get_bearer_token()
headers = create_headers(bearer_token)
user_id = get_user_id('PlayStation')

# Dates
start_time = datetime.now() - timedelta(hours=3)
end_time = datetime.now()

# Dictionary to store the data
data = {'date': [], 'text': []}

# Check if flag is true
while flag:
    
    response = client.get_users_mentions(id=user_id, start_time=start_time, end_time=end_time)
    result_count = response.meta['result_count']

    if 'next_token' in response.meta and total_tweets<100:
        # Save the token to use for next call
        next_token = response.meta['next_token']
        print("Next Token: ", next_token)

        if result_count is not None and result_count > 0 and next_token is not None:
            print("Start Date: ", start_time)
            for tweet in response.data:
                text = tweet['text'].replace('\n', ' ')
                data['date'].append(end_time)
                data['text'].append(text)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(2)   

    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            print("Start Date: ", start_time)
            for tweet in response.data:
                text = tweet['text'].replace('\n', ' ')
                data['date'].append(end_time)
                data['text'].append(text)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(2)

        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(2)
print("Total number of results: ", total_tweets)

Next Token:  7140dibdnow9c7btw421t2hajzxyzxp1r9ecnzwlpy3w6
Start Date:  2022-06-01 07:59:29.875591
Total # of Tweets added:  10
-------------------
Next Token:  7140dibdnow9c7btw421t2hajzxyzxp1r9ecnzwlpy3w6
Start Date:  2022-06-01 07:59:29.875591
Total # of Tweets added:  20
-------------------
Next Token:  7140dibdnow9c7btw421t2hajzxyzxp1r9ecnzwlpy3w6
Start Date:  2022-06-01 07:59:29.875591
Total # of Tweets added:  30
-------------------
Next Token:  7140dibdnow9c7btw421t2hajzxyzxp1r9ecnzwlpy3w6
Start Date:  2022-06-01 07:59:29.875591
Total # of Tweets added:  40
-------------------
Next Token:  7140dibdnow9c7btw421t2hajzxyzxp1r9ecnzwlpy3w6
Start Date:  2022-06-01 07:59:29.875591
Total # of Tweets added:  50
-------------------
Next Token:  7140dibdnow9c7btw421t2hajzxyzxp1r9ecnzwlpy3w6
Start Date:  2022-06-01 07:59:29.875591
Total # of Tweets added:  60
-------------------
Next Token:  7140dibdnow9c7btw421t2hajzxyzxp1r9ecnzwlpy3w6
Start Date:  2022-06-01 07:59:29.875591
Total # of Tw

In [31]:
data = pd.DataFrame.from_dict(data)
data.head()

,date,text
0,2022-06-01 10:59:29.875615,@PlayStation GTA
1,2022-06-01 10:59:29.875615,"@mtahmisian24 @gunsnroses2016 You do, Kids sho..."
2,2022-06-01 10:59:29.875615,"Venga va, yo también me sumo... Trofeos de Pl..."
3,2022-06-01 10:59:29.875615,Most Popular Game Videos on @PlayStation YouTu...
4,2022-06-01 10:59:29.875615,Apparently nightmare is an offensive word acco...


In [33]:
data.to_csv('gs://deep-feelings/test.csv')

In [39]:
# Check if the file exists
name = 'test.csv'   
storage_client = storage.Client()
bucket_name = 'deep-feelings'
bucket = storage_client.bucket(bucket_name)
stats = storage.Blob(bucket=bucket, name=name).exists(storage_client)
stats

True

# Cloud function

In [ ]:
# main.py

import time
import requests
import tweepy
from datetime import datetime, timedelta
import gcsfs
import pandas as pd

def get_user_id(user_name):
    url = f'https://api.twitter.com/2/users/by/username/{user_name}'
    response = requests.request("GET", url, headers = headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()['data']['id']

#Total number of tweets we collected from the loop
total_tweets = 0

# Inputs
max_results = 100 # Max tweets per time period
flag = True
next_token = None
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAImXdAEAAAAA7NWR%2FmlW%2B5jJhM5MRexIcuPo0OM%3D0TNtj03I70CUv0j72HBOeaibGeAmsIQI7cTHQlGDTGjk9NsRhr'
user_id = get_user_id('PlayStation')

# Dates
start_time = datetime.now() - timedelta(hours=3)
end_time = datetime.now()

# Dictionary to store the data
data = {'date': [], 'text': []}

# tweepy client
client = tweepy.Client(bearer_token)

# Check if flag is true
while flag:
    
    response = client.get_users_mentions(id=user_id, start_time=start_time, end_time=end_time)
    result_count = response.meta['result_count']

    if 'next_token' in response.meta and total_tweets<100:
        # Save the token to use for next call
        next_token = response.meta['next_token']
        print("Next Token: ", next_token)

        if result_count is not None and result_count > 0 and next_token is not None:
            print("Start Date: ", start_time)
            for tweet in response.data:
                text = tweet['text'].replace('\n', ' ')
                data['date'].append(end_time)
                data['text'].append(text)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(2)   

    # If no next token exists
    else:
        if result_count is not None and result_count > 0:
            print("-------------------")
            print("Start Date: ", start_time)
            for tweet in response.data:
                text = tweet['text'].replace('\n', ' ')
                data['date'].append(end_time)
                data['text'].append(text)
            total_tweets += result_count
            print("Total # of Tweets added: ", total_tweets)
            print("-------------------")
            time.sleep(2)

        #Since this is the final request, turn flag to false to move to the next time period.
        flag = False
        next_token = None
    time.sleep(2)
print("Total number of results: ", total_tweets)

data = pd.DataFrame.from_dict(data)
data.to_csv('gs://deep-feelings/test.csv')

In [ ]:
# requirements.txt

'''
time
requests>=2.27.1
tweepy>=4.10.0
datetime
gcsfs>=2022.3.0
pandas>=1.3.5
'''